# Problem Description
**Task**<br>
VLSI or Very Large-Scale Integration is a process of creating an integrated circuit by combining thousands of transistors on a single chip. You want to design a single layer of an integrated circuit. You know exactly what modules will be used in this layer, and which of them should be connected by wires. The wires will be all on the same layer, but they cannot intersect with each other. Also, each wire can only be bent once, in one of two directions — to the left or to the right. If you connect two modules with a wire, selecting the direction of bending uniquely defines the position of the wire. Of course, some positions of some pairs of wires lead to intersection of the wires, which is forbidden. You need to determine a position for each wire in such a way that no wires intersect.
This problem can be reduced to 2-SAT problem — a special case of the SAT problem in which each clause contains exactly 2 variables. For each wire 𝑖, denote by 𝑥𝑖 a binary variable which takes value 1 ifthewireisbenttotherightand0ifthewireisbenttotheleft.Foreach𝑖,𝑥𝑖 mustbeeither0or1. Also, some pairs of wires intersect in some positions. For example, it could be so that if wire 1 is bent to the left and wire 2 is bent to the right, then they intersect. We want to write down a formula which is satisfied only if no wires intersect. In this case, we will add the clause (𝑥1 𝑂𝑅 𝑥2) to the formula which ensures that either 𝑥1 (the first wire is bent to the right) is true or 𝑥2 (the second wire is bent to the left) is true, and so the particular crossing when wire 1 is bent to the left AND wire 2 is bent to the right doesn’t happen whenever the formula is satisfied. We will add such a clause for each pair of wires and each pair of their positions if they intersect when put in those positions. Of course, if some pair of wires intersects in any pair of possible positions, we won’t be able to design a circuit. Your task is to determine whether it is possible, and if yes, determine the direction of bending for each of the wires.
<br><br>**Input Format**<br>
The input represents a 2-CNF formula. The first line contains two integers 𝑉 and 𝐶 — the number of variables and the number of clauses respectively. Each of the next 𝐶 lines contains two non-zero integers 𝑖 and 𝑗 representing a clause in the CNF form. If 𝑖 > 0, it represents 𝑥𝑖, otherwise if 𝑖 < 0, it represents 𝑥−𝑖, and the same goes for 𝑗. For example, a line “2 3” represents a clause (𝑥2 𝑂𝑅 𝑥3), line “1 -4” represents (𝑥1 𝑂𝑅 𝑥4), line “-1 -3” represents (𝑥1 𝑂𝑅 𝑥3), and line “0 2” cannot happen, because 𝑖 and 𝑗 must be non-zero.
<br><br>**Constraints**<br>
1≤𝑉, 𝐶≤1000000 ; −𝑉≤𝑖,𝑗≤𝑉 ; 𝑖,𝑗̸=0.
<br><br>**Output Format**<br>
If the 2-CNF formula in the input is unsatisfiable, output just the word “UNSATISFI- ABLE” (without quotes, using capital letters). If the 2-CNF formula in the input is satisfiable, output the word “SATISFIABLE” (without quotes, using capital letters) on the first line and the corresponding assignment of variables on the second line. For each 𝑥𝑖 in order from 𝑥1 to 𝑥𝑉 , output 𝑖 if 𝑥𝑖 = 1 or −𝑖 if 𝑥𝑖 = 0. For example, if a formula is satisfied by assignment 𝑥1 = 0,𝑥2 = 1,𝑥3 = 0,

In [ ]:
import copy
import sys
import threading
sys.setrecursionlimit(10**6)
threading.stack_size(2**26)

n, m = map(int, input().split())
clauses = [ list(map(int, input().split())) for i in range(m) ]

sccadj = [[] for i in range(2*n+1)]
def toImplication(clauses):
    global sccadj

    for x in clauses:
        if -x[0] > 0:
            sccadj[-x[0]].append(x[1])
        else:
            sccadj[n+x[0]].append(x[1])
        if -x[1] > 0:
            sccadj[-x[1]].append(x[0])
        else:
            sccadj[n+x[1]].append(x[0])

#Iterative Tarjan
Time = 0
sccs = []
sccsList = []
SatiCheck = True
disc = [-1 for i in range(2*n+1)]
st = []
#additional lists for result
resultList = [0 for i in range(1, n+1)]
resultCheck = [0 for i in range(n)]
cnt = 0
newscc = [0 for i in range(n+1)]

def findSCC(u_):
    global Time, sccadj, disc, SatiCheck, resultList, resultCheck, cnt, newscc
    work = [(u_, 0)]

    while work:
        u_, i = work[-1]
        del work[-1]
        if i == 0:
            disc[u_] = Time
            low[u_] = Time
            Time += 1
            st.append(u_)
            stackMember[u_] = True
        recurse = False

        for j in range(i, len(sccadj[u_])):
            v = sccadj[u_][j]

            if v>0:
                v_ = v
            else:
                v_ = n-v

            if disc[v_] == -1:
                work.append((u_, j+1))
                work.append((v_,0))
                recurse = True
                break
            elif stackMember[v_]:
                low[u_] = min(low[u_], disc[v_])

        if recurse:
            continue
        v_ = -1

        if disc[u_] == low[u_]:
            cnt += 1

            while v_ != u_:
                v_ = st.pop()

                if v_ > n:
                    v_scc = n-v_
                    if resultCheck[-v_scc-1] ==0:
                        resultCheck[-v_scc-1] = 1
                        resultList[-v_scc-1] = 1

                    if newscc[-v_scc] == cnt:
                        SatiCheck = False
                    newscc[-v_scc] = cnt

                else:
                    v_scc = v_
                    if resultCheck[v_scc-1] ==0:
                        resultCheck[v_scc-1] = 1

                    if newscc[v_scc] == cnt:
                        SatiCheck = False
                    newscc[v_scc] = cnt
                stackMember[v_] = False

        if work:
            v_ = u_
            u_, _ = work[-1]
            low[u_] = min(low[u_], low[v_])

low = [-1 for i in range(2*n+1)]
stackMember = [False]*(2*n+1)

def main():
    global sccadj, impleGraph, disc, low, Time, SatiCheck, resultList, cnt, newscc

    impleGraph = toImplication(clauses)

    for i in range(1, 2*n+1):
        if disc[i] == -1:
            findSCC(i)

        if SatiCheck == False:
            print('UNSATISFIABLE')
            return
    print('SATISFIABLE')
    print(" ".join(str(-i-1 if resultList[i] else i+1) for i in range(n)))

threading.Thread(target=main).start()